## Preliminary definitions

In [693]:
import numpy as np
import pandas as pd
import scipy 
import random
from itertools import product, permutations, combinations
from collections import Counter

In [694]:
f=4 #number of features
s=3 #number of cards per gset/options per feature
n_cards=s**f #total number of cards

dimensions=(s,) *f
empty_board=np.zeros(dimensions)

In [695]:
def add_tuples(*tuples):
    # Using zip to pair up the corresponding elements and sum them element-wise
    return tuple(sum(elements) for elements in zip(*tuples))

In [696]:
def create_indexed_nd_array(s,f):
    # Generate the grid of indices
    dimensions=(s,) *f
    grid = np.indices(dimensions)
    
    # Combine the indices into a single array with the desired shape
    # Transpose the grid to get the correct shape and then reshape
    indexed_array = np.moveaxis(grid, 0, -1)
    
    return indexed_array

### Random card generator

In [697]:
def card_gen(s,f):
    return tuple(random.randrange(s) for _ in range(f))

### All the cards

In [698]:
def all_cards(s,f):
    return list(product(*map(range, [s]*f)))

# all_cards(s,f)

### List of cards <--> filled board

In [699]:
def board_to_cards(board):
    # Input: np.array for board of ones (cards on the board) and zeros (cards not on the board)
    # Returns list of vectors of cards (tuples)
    card_inds=np.column_stack(np.where(board == 1)).tolist()
    card_inds=[tuple(_) for _ in card_inds]
    return card_inds

def cards_to_board(cards,s):
    # Input: list of vectors (tuples of coordinates) of cards; number of cards per gset 
    f=len(cards[0]) # Number of features determined from dimensionality of vectors
    # Returns 
    dimensions=(s,) *f
    board=np.zeros(dimensions)
    for card in cards:
        board[card] = 1
    return board

# a=np.asarray([[0,1,0,1,1],[1,1,1,1,1]])
# board_to_cards(a)
# cards_to_board([(1,1)],3)

### Whether group of cards is a gset

In [700]:
def a_gset(hand):
    s=len(hand)
    f=len(hand[0])
    for feat in range(f):
        diff_feats=len(set([hand[card][feat] for card in range(s)])) # Number of differing features
        if diff_feats!=1 and diff_feats!=s:
            return False
    return True


## All gsets

In [701]:
def gsets(s,f):
    dimensions=(s,) *f
    board_indices=create_indexed_nd_array(s,f)
    all_gsets=set()
    all_gsets_ls=[]
    
    # Generate all permutations of indices
    all_permutations = [list(permutations(range(s))) for i in range(f) ] 
    permuted_arrays = []

    # Generate all directions/lines that define gsets
    direcs=list(product(*map(range, [2]*f)))
    direcs=direcs[1:] #Remove 0,0,0,0 (no movement)
    n_direcs=len(direcs)

    origin=(0,) * f
    
    # Create all possible combinations of these permutations
    for perm_comb in product(*all_permutations):
        slices = tuple(np.ix_(*perm_comb))
        permuted_arr = board_indices[slices]
        permuted_arrays.append(permuted_arr)

        # Create gset for this permuted board for each direction/line
        for direc in direcs:
            gset=[tuple(permuted_arr[origin])]
            card=add_tuples(origin,direc)
#             print(card)
#             print(permuted_arr)
            for i in range(1,s):
                gset.append(tuple(permuted_arr[card]))
                card=add_tuples(card,direc)
            gset.sort()
            gset=tuple(gset)
            all_gsets.add(gset)
            all_gsets_ls.append(gset)
    
    return all_gsets,all_gsets_ls

# gsets(3,2)[0]

In [702]:
# Ignore

def permute_nd_array(arr):
    # Generate all permutations for each dimension
    dimensions = arr.shape
    all_permutations = [list(permutations(range(dim))) for dim in dimensions]

    permuted_arrays = []
    # Create all possible combinations of these permutations
    for perm_comb in product(*all_permutations):
        slices = tuple(np.ix_(*perm_comb))
        permuted_arr = arr[slices]
        permuted_arrays.append(permuted_arr)
    return permuted_arrays

# Example usage
arr_3d = np.array([[[1, 2,3], [4, 5,6],[7,8, 9]], [[10,11, 12],[13,14, 15],[16,17, 18]]])

### All gsets with a given card

In [703]:
# Inefficient solution: use gsets function and select the gsets containing the specified card

def cards_gsets(card,s,f):
    # Input: card as tuple; number of cards per gset/options per feature; number of features
    all_gsets=gsets(s,f)[0]
    gsets_w_card=[gset for gset in all_gsets if card in gset]
    return gsets_w_card

# cards_gsets((0,0,0,0), 3,4)

### All gsets in a board

In [706]:
def gsets_board(board):
    played_cards=board_to_cards(board)
    f=board.ndim
    s=len(board)
    possible_gs=[tuple(sorted(list(_))) for _ in combinations(played_cards, s)]
    all_gs=list(gsets(s,f)[0])
    gs_in=[_ for _ in possible_gs if _ in all_gs]
    return gs_in

## Gameplay: add a card until a gset

In [ ]:
def until_gset(s,f):
#     dimensions=(s,) *f
#     empty_board=np.zeros(dimensions)
#     board_indices=create_indexed_nd_array(s,f)
    all_card=all_cards(s,f)
    random.shuffle(all_card)
    
    all_gs=list(gsets(s,f)[0])
    
#     # Dictionary map: key 1 if on the board; 0.5 if not but when placed would yield a gset; 0 if not and wouldn't
#     # yield a gset
#     carded={}
#     for card in all_card:
#         carded[card]=0
    
#     # Counter map: inverse of carded dictionary; maps 1, 0.5, and 0 to relevant cards
#     counter_cards=Counter()
#     counter_cards[0]=all_card

    n_cards=s**f
    for i in range(n_cards-s):
        played_cards=all_card[:s+i]
        possible_gs=[tuple(sorted(list(_))) for _ in combinations(played_cards, s)]
        
        n=len(possible_gs)
        j=0
        print(i)
        while j<n:
            if possible_gs[j] in all_gs:
                return i+s
            j+=1
# until_gset(s,f)

## Data analysis

### Number of played cards until a gset

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Inefficient solution: finds all cards and all gsets for all n iterations
def count_cards(n,s,f):
    # Input: number of runs of game; number of cards per gset/options per feature; number of features
    count_dict=Counter()
    for i in range(n):
        count_dict[until_gset(s,f)]+=1
        print(i) #Disable
    return count_dict

In [ ]:
# More efficient: just reshuffles the deck 
def run_to_count(n,s,f):
#     dimensions=(s,) *f
#     empty_board=np.zeros(dimensions)
#     board_indices=create_indexed_nd_array(s,f)
    all_card=all_cards(s,f)
    all_gs=list(gsets(s,f)[0])
    
#     # Dictionary map: key 1 if on the board; 0.5 if not but when placed would yield a gset; 0 if not and wouldn't
#     # yield a gset
#     carded={}
#     for card in all_card:
#         carded[card]=0
    
#     # Counter map: inverse of carded dictionary; maps 1, 0.5, and 0 to relevant cards
#     counter_cards=Counter()
#     counter_cards[0]=all_card
    count_dict=Counter()
    n_cards=s**f
    for go in range(n):
        random.shuffle(all_card)
        i=0
        print(go)
        while i<n_cards-s:
            played_cards=all_card[:s+i]
            possible_gs=[tuple(sorted(list(_))) for _ in combinations(played_cards, 3)]

            n_pos=len(possible_gs)
            j=0
            while j<n_pos:
                if possible_gs[j] in all_gs:
                    count_dict[i+s]+=1
                    j=n_pos
                    i=n_cards
                j+=1
            i+=1
    return count_dict

In [ ]:
n_tries=1000
# n_counter=count_cards(n_tries,s,f)
n_counter=run_to_count(n_tries,s,f)

In [ ]:
df = pd.DataFrame.from_dict(n_counter,orient='index')
# df.plot(kind='bar', legend=False)
df.sort_index(inplace=True)
  
plt.hist(n_counter)
plt.xlabel("Number of cards until first gset")
plt.ylabel("Counts")
plt.show()
# df

# plt.bar(indexes, values, width)
# plt.xticks(indexes + width * 0.5, labels)
# plt.show()